In [3]:
from __future__ import print_function
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, LabelSet, HoverTool
import argparse
import numpy as np
from numpy.linalg import norm
from MulticoreTSNE import MulticoreTSNE as TSNE

# parser = argparse.ArgumentParser()
# parser.add_argument('-vec', action='store', dest='vectorfile', required=True,
#                     help='Path of model.vec generated by fasttext')
# parser.add_argument('-p', action='store', type=int, dest='perplexity',
#                     default=20, help='Perplexity, usually between 20 to 50')
# parser.add_argument('-topK', action='store', type=int, dest='topK',
#                     default=500, help='show topK words with largest embedding norm')

vectorfile = '/data/szr207/dataset/ArqMath/ft.models/arqmath.vec'
topK = 500
perplexity = 10

def load_data(vectorfile, topK=500):
    words = []
    with open(vectorfile) as fin:
        next(fin)  # skip first line containing word_num and embedding_dim
        for line in fin:
            line = line.strip('\n').split()
            words.append((line[0], line[1:]))
    print('Loaded {} words, embedding dim = {}'.format( len(words), len(words[0][1])))
    words = sorted(words, key=lambda x: norm(x[1]), reverse=True)

    labels, embeddings = [], []
    for word in words[:topK]:
        labels.append(word[0])
        embeddings.append(np.array(word[1]))
    return labels, np.array(embeddings)

def plot_embedding():
#     r = parser.parse_args()
#     for arg in vars(r):
#         print('%-10s =' % arg, getattr(r, arg))

    print('Loading data...')
    words, embeddings = load_data(vectorfile, topK)
    tsne = TSNE(n_jobs=4, perplexity=perplexity)
    Y = tsne.fit_transform(embeddings)
    source = ColumnDataSource(data=dict(x=Y[:,0], y=Y[:,1], words=words))
    hover = HoverTool(tooltips=[("word", "@words")])

    TOOLS = ['pan', 'wheel_zoom', 'reset', hover]
    p = figure(title='t-SNE embedding of top ' + str(len(Y)) + ' words',
            plot_width=1200, plot_height=720, tools=TOOLS)
    p.scatter(x='x', y='y', size=8, source=source)
    labels = LabelSet(x='x', y='y', text='words', level='glyph',
            x_offset=5, y_offset=5, source=source, render_mode='canvas')
    p.add_layout(labels)
    show(p)

plot_embedding()

Loading data...
Loaded 481575 words, embedding dim = 100


ValueError: could not convert string to float: 'see'

In [1]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import os 

root_dir = "/data/szr207/dataset/ArqMath/jsons/"

docs_raw = [line.strip() for line in open(os.path.join(root_dir,'questions.dump.txt'), 'r')]


tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(2477506, 50029)


In [2]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

/data/szr207/conda/envs/tf-gpu/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1813: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  UserWarning)


(2477506, 50029)


In [7]:
# lda_tf = LatentDirichletAllocation(n_components=20, random_state=0)
# lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=25, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=25, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [8]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/data/szr207/conda/envs/tf-gpu/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
17    -0.161574 -0.000959       1        1  5.759779
1     -0.176365 -0.065742       2        1  5.448826
13     0.081792  0.024148       3        1  5.291478
7      0.070404  0.063204       4        1  5.290942
12    -0.062712 -0.075303       5        1  5.195694
18     0.039308 -0.178974       6        1  4.981014
24     0.041583 -0.035439       7        1  4.896745
4      0.120315 -0.107746       8        1  4.755871
19    -0.151801 -0.101201       9        1  4.416497
8      0.139474 -0.079794      10        1  4.376401
9      0.042225  0.148269      11        1  4.163201
15     0.039327  0.042730      12        1  4.013800
14     0.049789 -0.091506      13        1  3.991468
20    -0.080296  0.072746      14        1  3.988579
22    -0.103581 -0.022139      15        1  3.955463
0     -0.030823  0.118318      16        1  3.808677
10     0.000405  0.019274      17        1  3.793530
16    -0.050968  0.029004      18        1  3.537612
3     -0.122052 -0.056479      19        1  3.366671
5      0.005802  0.098088      20        1  2.929806
6      0.100513 -0.097157      21        1  2.843111
23     0.126270  0.013259      22        1  2.577149
21    -0.025517  0.175396      23        1  2.500522
2      0.035876  0.084805      24        1  2.113771
11     0.072609  0.023199      25        1  2.003394, topic_info=      Category          Freq        Term         Total  loglift  logprob
20163  Default  37986.000000        hint  37986.000000  30.0000  30.0000
17154  Default  29324.000000        frac  29324.000000  29.0000  29.0000
26942  Default  27346.000000      matrix  27346.000000  28.0000  28.0000
1156   Default  14632.000000       align  14632.000000  27.0000  27.0000
21983  Default  18173.000000  inequality  18173.000000  26.0000  26.0000
...        ...           ...         ...           ...      ...      ...
25271  Topic25   1209.654085         let  58151.260544   0.0376  -5.0666
9539   Topic25    950.365980     correct  17065.735751   1.0223  -5.3079
7056   Topic25    833.817831      choose  10196.146700   1.4066  -5.4387
38733  Topic25    861.072020      second  17669.494644   0.8889  -5.4065
47370  Topic25    880.390758         use  33094.148944   0.2836  -5.3844

[2175 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
70         3  0.008983   abc
70         4  0.001123   abc
70         5  0.010106   abc
70         6  0.022458   abc
70         7  0.143729   abc
...      ...       ...   ...
49927     23  0.385921  zeta
49934      1  0.014327   zfc
49934      2  0.001102   zfc
49934      5  0.978638   zfc
49934      6  0.004408   zfc

[19835 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 2, 14, 8, 13, 19, 25, 5, 20, 9, 10, 16, 15, 21, 23, 1, 11, 17, 4, 6, 7, 24, 22, 3, 12])

In [11]:
lda_tfidflda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

/data/szr207/conda/envs/tf-gpu/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
17    -0.161574 -0.000959       1        1  5.759779
1     -0.176365 -0.065742       2        1  5.448826
13     0.081792  0.024148       3        1  5.291478
7      0.070404  0.063204       4        1  5.290942
12    -0.062712 -0.075303       5        1  5.195694
18     0.039308 -0.178974       6        1  4.981014
24     0.041583 -0.035439       7        1  4.896745
4      0.120315 -0.107746       8        1  4.755871
19    -0.151801 -0.101201       9        1  4.416497
8      0.139474 -0.079794      10        1  4.376401
9      0.042225  0.148269      11        1  4.163201
15     0.039327  0.042730      12        1  4.013800
14     0.049789 -0.091506      13        1  3.991468
20    -0.080296  0.072746      14        1  3.988579
22    -0.103581 -0.022139      15        1  3.955463
0     -0.030823  0.118318      16        1  3.808677
10     0.000405  0.019274      17        1  3.793530
16    -0.050968  0.029004      18        1  3.537612
3     -0.122052 -0.056479      19        1  3.366671
5      0.005802  0.098088      20        1  2.929806
6      0.100513 -0.097157      21        1  2.843111
23     0.126270  0.013259      22        1  2.577149
21    -0.025517  0.175396      23        1  2.500522
2      0.035876  0.084805      24        1  2.113771
11     0.072609  0.023199      25        1  2.003394, topic_info=      Category          Freq        Term         Total  loglift  logprob
20163  Default  37986.000000        hint  37986.000000  30.0000  30.0000
17154  Default  29324.000000        frac  29324.000000  29.0000  29.0000
26942  Default  27346.000000      matrix  27346.000000  28.0000  28.0000
1156   Default  14632.000000       align  14632.000000  27.0000  27.0000
21983  Default  18173.000000  inequality  18173.000000  26.0000  26.0000
...        ...           ...         ...           ...      ...      ...
25271  Topic25   1209.654085         let  58151.260544   0.0376  -5.0666
9539   Topic25    950.365980     correct  17065.735751   1.0223  -5.3079
7056   Topic25    833.817831      choose  10196.146700   1.4066  -5.4387
38733  Topic25    861.072020      second  17669.494644   0.8889  -5.4065
47370  Topic25    880.390758         use  33094.148944   0.2836  -5.3844

[2175 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
70         3  0.008983   abc
70         4  0.001123   abc
70         5  0.010106   abc
70         6  0.022458   abc
70         7  0.143729   abc
...      ...       ...   ...
49927     23  0.385921  zeta
49934      1  0.014327   zfc
49934      2  0.001102   zfc
49934      5  0.978638   zfc
49934      6  0.004408   zfc

[19835 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 2, 14, 8, 13, 19, 25, 5, 20, 9, 10, 16, 15, 21, 23, 1, 11, 17, 4, 6, 7, 24, 22, 3, 12])